In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## OpenAI Functions: Classic Way

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.openai_functions import create_openai_fn_chain
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [3]:
MODEL_GPT = 'gpt-4o-mini'

In [4]:
# llm = ChatOpenAI()
llm = ChatOpenAI(model=MODEL_GPT)

In [5]:
prompt = ChatPromptTemplate.from_template(
    "Tell me the name of the captain of the team who won the soccer world cup of {year}?"
)

In [6]:
output_parser = JsonOutputFunctionsParser()

In [7]:
functions = [
    {
      "name": "player_full_name",
      "description": "Name and lastname of the player",
      "parameters": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string",
            "description": "The first name of the player"
          },
          "lastname": {
            "type": "string",
            "description": "The lastname of the player"
          }
        },
        "required": ["name", "lastname"]
      }
    }
  ]

In [8]:
chain = create_openai_fn_chain(
    functions=functions,
    llm=llm, 
    prompt=prompt,
    output_parser=output_parser
)
chain.predict(year="2010")

C:\Users\Pavel\AppData\Local\Temp\ipykernel_9356\1586263087.py:1: LangChainDeprecationWarning: The function `create_openai_fn_chain` was deprecated in LangChain 0.1.1 and will be removed in 1.0. Use :meth:`~create_openai_fn_runnable` instead.
  chain = create_openai_fn_chain(


{'name': 'Iker', 'lastname': 'Casillas'}

## OpenAI Functions: New LCEL Way

In [9]:
new_chain = (
    prompt 
    | llm.bind(function_call={"name": "player_full_name"}, functions=functions)
    | JsonOutputFunctionsParser()
)
new_chain.invoke({"year": "2010"})

{'name': 'Iker', 'lastname': 'Casillas'}